In [1]:
import pandas as pd

data = pd.read_csv("CP_count_PUMA.csv")
data

,smiles,1_1,10_19,10_20,100_277,100_278,104_291,105_355,105_369,105_361,...,9_17,9_18,90_256,91_257,92_258,93_259,InChIKey14,Cells_Number_Object_Number,Cells_Neighbors_NumberOfNeighbors_Adjacent,Cells_Parent_Nuclei
0,CCOC(=O)c1ccc(NC(=S)N2CCSC2c2ccc(OC)cc2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,JEFNGIWYLXETLU-UHFFFAOYSA-N,1.328125,0.339844,1.328125
1,O=C(CC1NC(=O)NC1=O)Nc1cccc2ccccc12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TYLHRPBDFWODNS-UHFFFAOYSA-N,2.503906,0.429688,2.503906
2,CC(Nc1nc(nc2ccccc12)N1CCCC1)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,HKGJEZIGDHFJFL-UHFFFAOYSA-N,-8.250000,-0.023438,-8.250000
3,CCCC(Oc1ccc(Br)cc1)c1nc2c3cc(OC)c(OC)cc3nc(S)n2n1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TXQUPVHAFRNVNR-UHFFFAOYSA-N,-7.804688,-0.304688,-7.796875
4,CC1=C(C(NC(=O)N1)c1ccc(F)cc1)C(=O)OCc1ccc2OCOc2c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,QCGHTRAZOATGMB-UHFFFAOYSA-N,-10.960938,-0.945312,-10.960938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,COC(=O)[C@H]1[C@H](CO)[C@H]2Cn3c(=O)c(\C=C\C)c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,HWXVAOXZBVDFCJ-PNJOTBKASA-N,-1.550781,-0.117188,-1.550781
16166,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@H](N(C)[C@@H]...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,RCRORROKKZWOHQ-ILYPDCDPSA-N,-0.835938,-0.191406,-0.835938
16167,C\C=C\c1ccc2n(C[C@@H]3[C@@H](CO)[C@@H](N(C)[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,RCRORROKKZWOHQ-QSXOYBCMSA-N,-3.835938,0.335938,-3.835938
16168,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@@H](C(=O)N[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,UURQJTAZHVTRJQ-MYKHNXKGSA-N,-3.300781,-0.117188,-3.300781


In [3]:
assays_list = data.columns.to_list()[1:-4]

In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# GPU Configuration
def configure_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            for device in physical_devices:
                tf.config.experimental.set_memory_growth(device, True)
            tf.config.experimental.set_visible_devices(physical_devices, 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            logging.info(f'{len(physical_devices)} Physical GPUs, {len(logical_gpus)} Logical GPUs')
        except RuntimeError as e:
            logging.error(f"GPU configuration error: {e}")
    else:
        logging.warning("No GPU devices found")

configure_gpu()

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

missing_value_indicator = -1

def check_data(data, name="Data"):
    """Check if the data contains NaN or infinite values."""
    if np.any(np.isnan(data)) or np.any(np.isinf(data)):
        logging.warning(f"{name} contains NaN or infinite values.")
    else:
        logging.info(f"{name} is clean.")

# Custom Loss Function to Ignore Missing Values
def custom_loss(y_true, y_pred):
    mask = tf.not_equal(y_true, missing_value_indicator)
    mask = tf.cast(mask, dtype=tf.float32)
    y_true_masked = tf.multiply(y_true, mask)
    y_pred_masked = tf.multiply(y_pred, mask)
    loss = tf.keras.losses.binary_crossentropy(y_true_masked, y_pred_masked)
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)  # Normalize by the number of available targets
    return loss

# Custom AUC function to ignore missing values
class MaskedAUC(tf.keras.metrics.AUC):
    def __init__(self, name='masked_auc', **kwargs):
        super(MaskedAUC, self).__init__(name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.not_equal(y_true, missing_value_indicator)
        mask = tf.cast(mask, dtype=tf.bool)
        y_true_masked = tf.boolean_mask(y_true, mask)
        y_pred_masked = tf.boolean_mask(y_pred, mask)
        super(MaskedAUC, self).update_state(y_true_masked, y_pred_masked, sample_weight)

# Build the Neural Network with Regularization and Batch Normalization
def build_model(input_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(output_dim, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss=custom_loss, metrics=[MaskedAUC()])
    return model

def load_and_preprocess_data(train_path, tt_path, data, assays_list):
    train_data = pd.read_csv(train_path)[["smiles"]]
    train_data = pd.merge(train_data, data, on="smiles")
    
    test_data = pd.read_csv(test_path)[["smiles"]]
    test_data = pd.merge(test_data, data, on="smiles")
    
    X_train = train_data[["Cells_Number_Object_Number", "Cells_Neighbors_NumberOfNeighbors_Adjacent", "Cells_Parent_Nuclei"]].values
    X_test = test_data[["Cells_Number_Object_Number", "Cells_Neighbors_NumberOfNeighbors_Adjacent", "Cells_Parent_Nuclei"]].values
    
    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    y_train = train_data[assays_list].replace(np.nan, missing_value_indicator).values
    y_test = test_data[assays_list].replace(np.nan, missing_value_indicator).values
    
    return X_train, X_test, y_train, y_test

def train_and_evaluate_model(fold, data, assays_list):
    train_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_train_scaff.csv"
    test_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_test_scaff.csv"
    
    X_train, X_test, y_train, y_test = load_and_preprocess_data(train_path, test_path, data, assays_list)
    
    # Check data for NaN or infinite values
    check_data(X_train, "X_train")
    check_data(y_train, "y_train")
    check_data(X_test, "X_test")
    check_data(y_test, "y_test")
    
    input_dim = X_train.shape[1]
    output_dim = y_train.shape[1]
    
    model = build_model(input_dim, output_dim)

    # Create a stratified split for the validation set using multilabel stratification
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, val_index in mskf.split(X_train, y_train):
        X_train_split, X_val_split = X_train[train_index], X_train[val_index]
        y_train_split, y_val_split = y_train[train_index], y_train[val_index]
        break  # Only need the first split

    # Define early stopping, learning rate scheduler, and model checkpoint
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_masked_auc', patience=10, restore_best_weights=True, mode='max')
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_masked_auc', factor=0.5, patience=5, mode='max')
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'best_model_fold_{fold}.h5', save_best_only=True, monitor='val_masked_auc', mode='max')

    # Train the Neural Network
    history = model.fit(X_train_split, y_train_split, epochs=100, batch_size=32, validation_data=(X_val_split, y_val_split),
                        callbacks=[early_stopping, lr_scheduler, model_checkpoint])

    # Evaluate the Model
    metrics = model.evaluate(X_test, y_test)
    for name, value in zip(model.metrics_names, metrics):
        logging.info(f"Fold {fold} - {name}: {value}")

    # Make predictions
    predictions = model.predict(X_test)
    
    predictions_df = pd.DataFrame(predictions, columns=assays_list)
    predictions_df.to_csv(f"predictions_fold_{fold}.csv", index=False)
    
    y_test_df = pd.DataFrame(y_test, columns=assays_list)
    y_test_df.to_csv(f"y_test_fold_{fold}.csv", index=False)

# Main execution
if __name__ == "__main__":
    for fold in range(5):
        train_and_evaluate_model(fold, data, assays_list)


INFO:root:8 Physical GPUs, 8 Logical GPUs


Num GPUs Available:  8


INFO:root:X_train is clean.
INFO:root:y_train is clean.
INFO:root:X_test is clean.
INFO:root:y_test is clean.


Epoch 1/100
324/324 [==============================] - 15s 18ms/step - loss: 0.0016 - masked_auc: 0.5312 - val_loss: 4.6238e-04 - val_masked_auc: 0.6658 - lr: 0.0010
Epoch 2/100
324/324 [==============================] - 5s 14ms/step - loss: 5.0648e-04 - masked_auc: 0.6368 - val_loss: 4.1849e-04 - val_masked_auc: 0.7251 - lr: 0.0010
Epoch 3/100
324/324 [==============================] - 5s 14ms/step - loss: 4.6883e-04 - masked_auc: 0.6771 - val_loss: 4.1386e-04 - val_masked_auc: 0.7305 - lr: 0.0010
Epoch 4/100
324/324 [==============================] - 5s 15ms/step - loss: 4.5668e-04 - masked_auc: 0.6923 - val_loss: 4.1186e-04 - val_masked_auc: 0.7363 - lr: 0.0010
Epoch 5/100
324/324 [==============================] - 5s 15ms/step - loss: 4.5011e-04 - masked_auc: 0.6998 - val_loss: 4.1035e-04 - val_masked_auc: 0.7395 - lr: 0.0010
Epoch 6/100
324/324 [==============================] - 5s 14ms/step - loss: 4.4418e-04 - masked_auc: 0.7054 - val_loss: 4.1054e-04 - val_masked_auc: 0.7378 - 

INFO:root:Fold 0 - loss: 0.0004013214784208685
INFO:root:Fold 0 - masked_auc: 0.7566918134689331


...

102/102 [==============================] - 1s 3ms/step


INFO:root:X_train is clean.
INFO:root:y_train is clean.
INFO:root:X_test is clean.
INFO:root:y_test is clean.


Epoch 1/100
323/323 [==============================] - 12s 16ms/step - loss: 0.0016 - masked_auc: 0.5361 - val_loss: 4.6357e-04 - val_masked_auc: 0.6577 - lr: 0.0010
Epoch 2/100
323/323 [==============================] - 4s 14ms/step - loss: 5.0537e-04 - masked_auc: 0.6290 - val_loss: 4.3220e-04 - val_masked_auc: 0.7115 - lr: 0.0010
Epoch 3/100
323/323 [==============================] - 4s 14ms/step - loss: 4.6406e-04 - masked_auc: 0.6755 - val_loss: 4.1765e-04 - val_masked_auc: 0.7398 - lr: 0.0010
Epoch 4/100
323/323 [==============================] - 4s 13ms/step - loss: 4.5029e-04 - masked_auc: 0.6922 - val_loss: 4.1653e-04 - val_masked_auc: 0.7381 - lr: 0.0010
Epoch 5/100
323/323 [==============================] - 4s 14ms/step - loss: 4.4314e-04 - masked_auc: 0.7055 - val_loss: 4.1398e-04 - val_masked_auc: 0.7426 - lr: 0.0010
Epoch 6/100
323/323 [==============================] - 4s 14ms/step - loss: 4.3771e-04 - masked_auc: 0.7126 - val_loss: 4.1224e-04 - val_masked_auc: 0.7460 - 

INFO:root:Fold 4 - loss: 0.00041812274139374495
INFO:root:Fold 4 - masked_auc: 0.7541331648826599


102/102 [==============================] - 1s 3ms/step
